# Lab 7, Module 1: Applying Filters to Real Images

**Estimated time:** 15 minutes

---

## **Opening: From Tiny Arrays to Real Images**

In Module 0, you learned the basic concept of convolution using a tiny 5×5 image.

**Now it's time to see convolution in action on real photographs!**

You'll apply classic computer vision filters that have been used for decades:
- **Blur** (smoothing, noise reduction)
- **Sharpen** (edge enhancement)
- **Sobel edge detection** (finding boundaries)
- **Laplacian edge detection** (finding all edges at once)

These same operations power modern CNNs—the difference is that **CNNs learn their own filters** through training, while these classic filters were designed by hand.

### **Connection to Lab 6**

Remember Lab 6's saliency maps?
- **Lab 6:** Showed **WHERE** a model looks
- **Lab 7 (today):** Shows **WHAT FEATURES** a model extracts

Edge detection and texture filters are exactly what the early layers of CNNs learn to detect!

---

## 🖼️ **Sample Images**

We'll work with two sample images:
1. **edges_demo.jpg** – Architecture with clear edges (vertical, horizontal, diagonal)
2. **textures_demo.jpg** – Natural scene with textures (fur, grass, leaves)

You'll also have the option to **upload your own image** and see how filters transform it!

---

In [ ]:
# Setup: Install and import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files
from io import BytesIO
from PIL import Image
import requests

print("✅ Libraries imported successfully!")
print("\nReady to apply filters to real images!")

---

## 📥 **Load Sample Images**

We'll load two sample images for demonstration:
1. Architecture image (strong edges)
2. Nature image (textures)

---

In [ ]:
# For this demo, we'll create synthetic sample images
# In a real deployment, you'd load actual images from sample_images/

def create_sample_image_with_edges():
    """Create a simple geometric pattern with edges"""
    img = np.ones((256, 256), dtype=np.uint8) * 200
    
    # Add vertical stripes
    img[:, 50:80] = 50
    img[:, 100:130] = 50
    img[:, 150:180] = 50
    
    # Add horizontal stripe
    img[100:130, :] = 100
    
    # Add diagonal rectangle
    img[180:220, 180:220] = 30
    
    return img

def create_sample_image_with_textures():
    """Create an image with various textures"""
    img = np.ones((256, 256), dtype=np.uint8) * 128
    
    # Add random noise in different regions (simulating texture)
    # Region 1: Fine texture
    img[:128, :128] = np.random.randint(80, 120, (128, 128))
    
    # Region 2: Coarse texture
    img[:128, 128:] = np.random.randint(50, 100, (128, 128))
    
    # Region 3: Smooth with edges
    img[128:, :128] = 180
    img[128:, :128][::4, ::4] = 60  # Dots
    
    # Region 4: Gradient
    img[128:, 128:] = np.linspace(50, 200, 128).reshape(1, -1).repeat(128, axis=0).astype(np.uint8)
    
    return img

# Create sample images
edges_demo = create_sample_image_with_edges()
textures_demo = create_sample_image_with_textures()

# Display sample images
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(edges_demo, cmap='gray')
axes[0].set_title('Sample Image 1: Edges Demo\n(Geometric patterns with clear boundaries)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(textures_demo, cmap='gray')
axes[1].set_title('Sample Image 2: Textures Demo\n(Various texture patterns)', fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Sample images loaded!")
print("\nWe'll use these images to demonstrate different filters.")

---

## 🔧 **Classic Filters**

Let's define the filters we'll use:

### **1. Blur Filter (Averaging)**
```
[1  1  1]
[1  1  1]  ÷ 9
[1  1  1]
```
**Effect:** Averages nearby pixels, reduces noise

### **2. Sharpen Filter**
```
[ 0 -1  0]
[-1  5 -1]
[ 0 -1  0]
```
**Effect:** Emphasizes differences between pixels

### **3. Sobel Vertical Edge Detector**
```
[-1  0  1]
[-2  0  2]
[-1  0  1]
```
**Effect:** Detects vertical edges (dark-to-bright transitions)

### **4. Sobel Horizontal Edge Detector**
```
[-1 -2 -1]
[ 0  0  0]
[ 1  2  1]
```
**Effect:** Detects horizontal edges

### **5. Laplacian (All Edges)**
```
[ 0  1  0]
[ 1 -4  1]
[ 0  1  0]
```
**Effect:** Detects edges in all directions at once

---

In [ ]:
# Define filters
filters = {
    'Original': None,  # No filter (for comparison)
    
    'Blur (3x3)': np.ones((3, 3), dtype=np.float32) / 9,
    
    'Sharpen': np.array([[0, -1, 0],
                         [-1, 5, -1],
                         [0, -1, 0]], dtype=np.float32),
    
    'Sobel Vertical': np.array([[-1, 0, 1],
                                [-2, 0, 2],
                                [-1, 0, 1]], dtype=np.float32),
    
    'Sobel Horizontal': np.array([[-1, -2, -1],
                                  [0, 0, 0],
                                  [1, 2, 1]], dtype=np.float32),
    
    'Laplacian': np.array([[0, 1, 0],
                          [1, -4, 1],
                          [0, 1, 0]], dtype=np.float32)
}

print("✅ Filters defined!")
print(f"\nWe have {len(filters)} filters ready to apply.")

---

## 🎨 **Apply Filters to Edges Demo Image**

Let's see how each filter transforms the geometric pattern image!

---

In [ ]:
# Apply all filters to edges demo image
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, kernel) in enumerate(filters.items()):
    if kernel is None:
        # Original image
        result = edges_demo
    else:
        # Apply convolution
        result = cv2.filter2D(edges_demo, -1, kernel)
    
    # Plot result
    axes[idx].imshow(result, cmap='gray')
    axes[idx].set_title(name, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('Filter Effects on Edges Demo Image', fontsize=14, fontweight='bold', y=1.02)
plt.show()

print("\n" + "="*70)
print("OBSERVATIONS:")
print("="*70)
print("\n1. BLUR: Softens edges, reduces sharp transitions")
print("   - Vertical and horizontal stripes become less distinct")
print("   - Useful for noise reduction\n")

print("2. SHARPEN: Makes edges more pronounced")
print("   - Boundaries become crisper and more defined")
print("   - Can amplify noise if present\n")

print("3. SOBEL VERTICAL: Detects vertical edges ONLY")
print("   - Vertical stripes show strong response (bright lines)")
print("   - Horizontal stripe barely visible\n")

print("4. SOBEL HORIZONTAL: Detects horizontal edges ONLY")
print("   - Horizontal stripe shows strong response")
print("   - Vertical stripes barely visible\n")

print("5. LAPLACIAN: Detects ALL edges")
print("   - Both vertical and horizontal edges highlighted")
print("   - More general-purpose edge detector")
print("="*70)

---

## 🎨 **Apply Filters to Textures Demo Image**

Now let's see how the same filters respond to textures!

---

In [ ]:
# Apply all filters to textures demo image
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, kernel) in enumerate(filters.items()):
    if kernel is None:
        # Original image
        result = textures_demo
    else:
        # Apply convolution
        result = cv2.filter2D(textures_demo, -1, kernel)
    
    # Plot result
    axes[idx].imshow(result, cmap='gray')
    axes[idx].set_title(name, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('Filter Effects on Textures Demo Image', fontsize=14, fontweight='bold', y=1.02)
plt.show()

print("\n" + "="*70)
print("OBSERVATIONS:")
print("="*70)
print("\n1. BLUR: Smooths fine textures")
print("   - Random noise patterns become less visible")
print("   - Different texture regions become more uniform\n")

print("2. SHARPEN: Emphasizes texture details")
print("   - Fine patterns become more visible")
print("   - Boundaries between texture regions enhanced\n")

print("3. SOBEL VERTICAL: Highlights vertical texture patterns")
print("   - Gradient region shows strong response")
print("   - Detects boundaries between texture regions\n")

print("4. SOBEL HORIZONTAL: Highlights horizontal texture patterns")
print("   - Different response pattern than vertical")
print("   - Shows how texture orientation matters\n")

print("5. LAPLACIAN: Reveals all texture boundaries")
print("   - Clear separation between the four quadrants")
print("   - Highlights both edges and fine texture patterns")
print("="*70)

---

## 🎨 **Interactive: Design Your Own Filter!**

Now it's your turn! Design a custom 3×3 filter and see what it does.

**Try these experiments:**
- All 1s: `[[1,1,1],[1,1,1],[1,1,1]]` (strong blur)
- All 0s except center: `[[0,0,0],[0,9,0],[0,0,0]]` (strong center emphasis)
- Diagonal pattern: `[[2,0,0],[0,2,0],[0,0,2]]` (diagonal edge detector)
- Your own creative pattern!

---

In [ ]:
# INTERACTIVE: Design your own 3x3 filter!
# Modify the values below

custom_filter = np.array([
    [0,  0,  0],   # Top row
    [0,  1,  0],   # Middle row
    [0,  0,  0]    # Bottom row
], dtype=np.float32)

# Apply to edges demo
custom_result_edges = cv2.filter2D(edges_demo, -1, custom_filter)

# Apply to textures demo
custom_result_textures = cv2.filter2D(textures_demo, -1, custom_filter)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Show custom filter
im = axes[0, 0].imshow(custom_filter, cmap='RdBu', vmin=-2, vmax=2)
axes[0, 0].set_title('Your Custom Filter', fontsize=12, fontweight='bold')
axes[0, 0].set_xticks(range(3))
axes[0, 0].set_yticks(range(3))
axes[0, 0].grid(True, color='black', linewidth=0.5)
# Add values
for i in range(3):
    for j in range(3):
        axes[0, 0].text(j, i, f'{custom_filter[i,j]:.1f}', 
                       ha='center', va='center', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=axes[0, 0], fraction=0.046)

# Show original images
axes[0, 1].imshow(edges_demo, cmap='gray')
axes[0, 1].set_title('Original: Edges Demo', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(textures_demo, cmap='gray')
axes[0, 2].set_title('Original: Textures Demo', fontsize=12, fontweight='bold')
axes[0, 2].axis('off')

# Empty space
axes[1, 0].axis('off')

# Show results
axes[1, 1].imshow(custom_result_edges, cmap='gray')
axes[1, 1].set_title('After Your Filter: Edges', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(custom_result_textures, cmap='gray')
axes[1, 2].set_title('After Your Filter: Textures', fontsize=12, fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.suptitle('Your Custom Filter Results', fontsize=14, fontweight='bold', y=1.02)
plt.show()

print("\n💡 TIP: Try modifying the custom_filter values above and re-run this cell!")
print("\nExperiment ideas:")
print("- Make a strong blur: [[1,1,1],[1,1,1],[1,1,1]] / 9")
print("- Make a diagonal edge detector: [[2,0,0],[0,0,0],[0,0,-2]]")
print("- Create asymmetric patterns and see what happens!")

---

## 📤 **Upload Your Own Image (Optional)**

Want to try these filters on your own photos? Upload an image here!

**Good images to try:**
- Photos with clear edges (buildings, books, windows)
- Nature photos with textures (trees, grass, animals)
- Portraits (to see how filters detect facial features)

---

In [ ]:
# Upload your own image
print("📤 Click 'Choose Files' and select an image from your computer...\n")

uploaded = files.upload()

if uploaded:
    # Get the first uploaded file
    filename = list(uploaded.keys())[0]
    
    # Read image
    img_bytes = uploaded[filename]
    img = Image.open(BytesIO(img_bytes))
    
    # Convert to grayscale numpy array
    img_gray = np.array(img.convert('L'))
    
    # Resize if too large (for faster processing)
    if img_gray.shape[0] > 512 or img_gray.shape[1] > 512:
        img_pil = Image.fromarray(img_gray)
        img_pil.thumbnail((512, 512))
        img_gray = np.array(img_pil)
    
    print(f"✅ Image loaded: {filename}")
    print(f"   Size: {img_gray.shape[0]} × {img_gray.shape[1]} pixels")
    
    # Apply all filters
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, (name, kernel) in enumerate(filters.items()):
        if kernel is None:
            result = img_gray
        else:
            result = cv2.filter2D(img_gray, -1, kernel)
        
        axes[idx].imshow(result, cmap='gray')
        axes[idx].set_title(name, fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'Filter Effects on Your Image: {filename}', fontsize=14, fontweight='bold', y=1.02)
    plt.show()
    
else:
    print("No image uploaded. Skipping this section.")

---

## 🔗 **Connection to CNNs: Learned vs. Hand-Designed Filters**

**What you just saw:**
- **Hand-designed filters** (Sobel, Laplacian, blur, sharpen)
- Filters created by computer vision researchers based on mathematical principles
- Work well for specific tasks (edge detection, noise reduction)

**What CNNs do differently:**
- **Learn filters automatically** through training
- Start with random filter values
- Gradient descent adjusts filter values to minimize loss
- Final filters detect patterns that are actually useful for the task

**Comparison:**

| Hand-Designed Filters | CNN Learned Filters |
|-----------------------|---------------------|
| Fixed patterns (Sobel, etc.) | Adapt to specific dataset |
| General-purpose | Task-specific |
| Work moderately well everywhere | Work excellently on training domain |
| 5-10 filters typically | 32-512 filters per layer! |

**The power of deep learning:**
- Early CNN layers learn edge detectors (similar to Sobel)
- Mid CNN layers learn texture detectors
- Deep CNN layers learn object part detectors
- **All automatically, without human design!**

---

## 🔗 **Connection to Lab 3: Activation Functions**

Remember Lab 3 where you learned about ReLU, sigmoid, and other activation functions?

**In CNNs, convolution and activation work together:**

```
Input Image
   ↓
Convolution (linear operation: multiply-and-add)
   ↓
Activation Function (nonlinear: ReLU, sigmoid, etc.)
   ↓
Feature Map (output)
```

**Why both are needed:**
- **Convolution alone:** Can only detect linear patterns
- **Activation function:** Adds nonlinearity, enables complex decisions
- **Together:** Can detect complex patterns (curves, textures, shapes)

**Example:**
```python
# Typical CNN layer
feature_map = conv2d(image, filter)  # Convolution
activated = relu(feature_map)        # Activation
```

**Just like Lab 3:**
- Lab 3: Dense layer + ReLU → Warped feature space
- Lab 7: Conv layer + ReLU → Edge/texture/shape detectors

---

## 📝 **Questions (Q5-Q9)**

Record your answers in the **Answer Sheet**.

---

### **Q5. After applying the blur filter, what changed about the image? Why might blur be useful in image processing?**

*Hint: Look at the edges—did they become sharper or softer? Think about noise reduction.*

**Record your answer in the Answer Sheet.**

---

### **Q6. What happened when you applied the Sobel vertical edge detector? Which parts of the image were highlighted?**

*Hint: Compare vertical vs. horizontal edges. Why does the vertical edge detector respond strongly to vertical stripes but not horizontal ones?*

**Record your answer in the Answer Sheet.**

---

### **Q7. Compare the sharpened image to the original. What features became more pronounced?**

*Hint: Look at boundaries and transitions. How does sharpen differ from edge detection?*

**Record your answer in the Answer Sheet.**

---

### **Q8. Design your own 3×3 filter in the interactive section. What effect did it have? Record your filter values and describe the result.**

*Record your custom filter and observations:*
```
Your filter:
[ __ __ __ ]
[ __ __ __ ]
[ __ __ __ ]

Effect observed: ___________
```

**Record your answer in the Answer Sheet.**

---

### **Q9. Why do you think edge detection is important for object recognition?**

*Hint: Think about what defines shapes. How do edges relate to object boundaries?*

**Record your answer in the Answer Sheet.**

---

## ✅ Module 1 Complete!

You now understand:
- **How to apply filters to real images** (blur, sharpen, edge detection)
- **What different filters detect** (edges, textures, boundaries)
- **How to design custom filters** and predict their effects
- **The connection to CNNs** (learned filters vs. hand-designed filters)
- **The role of activation functions** (convolution + nonlinearity = powerful feature extraction)

**Key insight:**
> CNNs automatically learn filters that are optimized for the task—starting with edge detectors in early layers and building up to complex pattern detectors in deeper layers.

**Ready to see inside a real CNN?**

Move on to **Module 2: Visualizing CNN Feature Maps**, where you'll load a pretrained MobileNetV2 model and visualize what its layers actually detect!

---